### Docker commands

__Docker run__

- `docker run <image_name>`
    - To run a docker container
    - Equivalent to `docker pull` + `docker start`
    - We can specify the configuration of the container to be created
- `docker run -d <image_name>`
    - To run a docker container in a detached mode
- `docker run --name <new_container_name> <image_name>`
    - To run a container and give it a name (otherwise assigned randomly)
- `docker run -p<port_to_bind_to>:<container_port> <image_name>`
    - To bind a container with a specific container_port to a specific port on system
- `docker run --net <network_name> <image_name>`
    - To run docker container inside a docker network (Not sure...) 

---

__Docker pull__

- `docker pull <image_name>`
    - To pull a docker image

---

__Docker start__

- `docker start <container_id>`
    - To start a docker container
    - We don't need to specify the configuration, we can directly start an existing container

---

__Docker stop__

- `docker stop <container_id>`
    - To stop a docker container

---

__Docker tag__

- `docker tag <old_image_name> <new_image_name>`
    - Creates a copy of an existing docker image with the new name

---

__Docker images__

- `docker images`
    - To list docker images

---

__Docker ps__

- `docker ps`
    - To list running containers
- `docker ps -a`
    - To list all (running/not running) images

---

__Docker remove container__

- `docker rm <container_id>`
    - To delete a docker container

---

__Docker remove image__

- `docker rmi <image_id>`
    - To delete a docker image
- To delete a docker image, first stop the corresponding container, then delete the container and then delete the image

---

__Docker logs__

- `docker logs <container_id>`
    - To get logs of a container

---

__Docker exec__

- `docker exec -it <container_id> <command>`
    - To execute a command in docker container
- `docker exec -it <container_id> /bin/bash`
    - To start the bash terminal of a container in interactive mode
- `docker exec -it <container_id> /bin/sh`
    - To start the shell terminal of a container in interactive mode
- Not all containers have bash or shell terminal

---

__Docker network__

- `docker network ls`
    - To display list of existing networks
- `docker network create <network_name>`
    - To create a new docker network

---

__Docker compose__

- Docker compose takes care of creating a common network, so we don't have to specify a network name
- `docker-compose -f <config_file_name> up`
    - To start docker containers in the compose file
    - It automatically creates a network
- `docker-compose -f <config_file_name> down`
    - To shut down docker containers in the compose file
    - It automatically removes the created network

`docker-compose.yaml`

```
version: '3'
services:
    <container_name>
        image: <image_name>
        ports:
            - 8080:8080 (<-- just an example)
        environment:
            - <variable_1>: <value_1>
            - <variable_2>: <value_2>
            .
            .
            .
    <container_name>
        image: <image_name>
        ports:
            - 2027:2027 (<-- just an example)
        volumes:
            - <name>:<container_path>
        environment:
            - <variable_1>: <value_1>
            - <variable_2>: <value_2>
            . . .
```

__Docker build__

- A Docker File is a Blueprint for creating docker images.
- A docker file has to be named exactly this: `Dockerfile`

`Dockerfile`

```
FROM <image_name>  # Base image on top of which we want to build another image
ENV var1=val1 \
    var2=val2 \
    ...  # More env variables
USER <user>
WORKDIR <work_dir_path>  # Define the working directory of a Docker container at any given time
RUN mkdir -p /home/app  # Any Linux Commands here, will apply to the container environment
COPY [--chown=<user>:<group>] <host_path> <container_path>  # Executed locally, all files copied to the container environment
CMD ["node", "server.js"]  # This is an entrypoint command. Different from RUN \
                           # since we can have multiple RUN commands but CMD is used just once. \
                           # It specifies the default program that will execute \
                           # once the container runs
```

Example:

```
FROM node
ENV MONGO_USERNAME=admin \
    MONGO_PASSWORD=password \
    ...  # More env variables
RUN mkdir -p /home/app
COPY . /home/app
CMD ["node", "server.js"]
```

- We don't really need to copy `Dockerfile` and `docker-compose.yaml` files in the docker image:
    - Create a folder "app" within the project and move all the project files to this folder
    - Then use the command `COPY ./app /home/app` to copy all the project files to the docker image
- `COPY` command gets executed on the host
- `RUN` command gets executed on the docker container
- `docker build -t <new_image_name>:<new_image_tag/version> <project_location>`
    - Build a new docker image from a project


__Docker volumes__

- We can do it using `docker run` or `docker-compose`
- Three types:
    - Host Volumes
        - `docker run -v <host_directory>:<container_directory>`:
            - We decide where on the host file system the reference is made
    - Anonymous Volumes
        - `docker run -v <container_directory>`:
            - Host Directory automatically created by Docker
            - For each container, a folder is created that gets mounted
    - Named Volumes (* - Most common)
        - `docker run -v <name>:<container_directory>`:
            - An improvement over Anonymous Volumes
            - Specifies the name of the folder on the host file system
            - We can reference the volume by name
- The paths of named and anonymous volumes on the local system differ based on the OS
    - On Windows, it's `C:\ProgramData\docker\volumes`
    - On Linux, it's `/var/lib/docker/volumes`
    - On Mac, it's `/var/lib/docker/volumes`

`docker-compose.yaml`

```
version: '3'
services:
    <container_name>
        image: <image_name>
        ports:
            - 8080:8080 (<-- just an example)
        volumes:
            - <volume_name>:<data_path_in_container>  # The path differs for different databases \
                                                      # (ex., for MongoDB, it's "/data/db")
        . . .
    . . .

volumes:  # List all named volumes here
    volume_name-1:
        driver: local  # To create volume on the local file system
    volume_name-2
    . . .
```